# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
aux = data.groupby(by = ["ProductName","CustomerID",], as_index = False).sum()
aux.head()

,ProductName,CustomerID,SalesID,ProductID,Quantity
0,Anchovy Paste - 56 G Tube,639,618381,131,1
1,Anchovy Paste - 56 G Tube,883,6544621,131,1
2,Anchovy Paste - 56 G Tube,891,408243,131,1
3,Anchovy Paste - 56 G Tube,2566,313177,131,1
4,Anchovy Paste - 56 G Tube,2902,91061,131,1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
prd_cus_mtrx = pd.pivot_table(data = aux,values="Quantity", index = "CustomerID" , columns = ["ProductName"]).fillna(0)
prd_cus_mtrx = prd_cus_mtrx.div(prd_cus_mtrx.sum(axis=1), axis=0)
prd_cus_mtrx.T.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.016393,0.0,0.0,...,0.000000,0.015385,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.014493,0.015385,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
Appetizer - Mushroom Tart,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.016393,0.0,0.0,...,0.014493,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.012195,0.0
Appetizer - Sausage Rolls,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.013889,0.014706,0.014286,0.0,0.012195,0.0
Apricots - Dried,0.015873,0.0,0.0,0.0,0.013699,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.015385,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
similarity = squareform(pdist(prd_cus_mtrx, metric = "euclidean"))
similarity = pd.DataFrame(1/(1+similarity), columns= prd_cus_mtrx.index, index= prd_cus_mtrx.index)
similarity.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
top_5 = similarity.iloc[0].sort_values(ascending = False)[1:].head()
top_5.index

Int64Index([60862, 27672, 6001, 79458, 33759], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
prod_5_top = aux[aux.CustomerID.isin(top_5.index)]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
top5_products = prod_5_top.groupby(by = "ProductName").sum().sort_values(by = "Quantity", ascending = False)
top5_products.reset_index(inplace=True)
top5_products = top5_products[["ProductName", "Quantity"]]
top5_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
merge = pd.merge(top5_products,pd.DataFrame(prd_cus_mtrx.T[33]), on = "ProductName")
merge.columns = ["ProductName", "Quantity", "Purchased"]
recomendation = merge[merge["Purchased"]==0].sort_values(by = "Quantity", ascending = False)[:5]
recomendation

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
all_recomendations = {}
customers = list(set(data.CustomerID))
for customer in customers:
    top_5 = similarity.loc[customer].sort_values(ascending = False)[1:].head()
    prod_5_top = aux[aux.CustomerID.isin(top_5.index)]
    top5_products = prod_5_top.groupby(by = "ProductName").sum().sort_values(by = "Quantity", ascending = False)
    top5_products.reset_index(inplace=True)
    top5_products = top5_products[["ProductName", "Quantity"]]
    merge = pd.merge(top5_products,pd.DataFrame(prd_cus_mtrx.T[33]), on = "ProductName")
    merge.columns = ["ProductName", "Quantity", "Purchased"]
    recomendation = merge[merge["Purchased"]==0].sort_values(by = "Quantity", ascending = False)[:5]
    all_recomendations[customer] = list(recomendation["ProductName"])

In [12]:
all_recomendations[33]

['Wine - Redchard Merritt',
 'Bread - Calabrese Baguette',
 'Thyme - Lemon, Fresh',
 'Milk Powder',
 'Ecolab - Lime - A - Way 4/4 L']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [13]:
user_rec = pd.DataFrame.from_dict(all_recomendations, orient = "index").reset_index()
user_rec.columns = ["CustomerID", "Rec1", "Rec2", "Rec3", "Rec4", "Rec5"]
user_rec.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,83973,Beans - Kidney White,Rice - Jasmine Sented,Browning Caramel Glace,Tea - Herbal Sweet Dreams,Cake - Cake Sheet Macaroon
1,59399,Anchovy Paste - 56 G Tube,"Nut - Pistachio, Shelled",Bread Fig And Almond,Milk - 2%,Apricots Fresh
2,92168,Scallops - 10/20,Bouq All Italian - Primerba,Pastry - Choclate Baked,Olive - Spread Tapenade,Tea - Jasmin Green
3,49159,Wine - White Cab Sauv.on,Oil - Shortening - All - Purpose,"Bread - Roll, Soft White Round",Mussels - Cultivated,Cinnamon Buns Sticky
4,18441,Scallops - Live In Shell,Milk - 2%,Mussels - Cultivated,Whmis - Spray Bottle Trigger,Beef - Ground Medium


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [14]:
similarity = squareform(pdist(prd_cus_mtrx, metric = "cosine"))
similarity = pd.DataFrame(1/(1+similarity), columns= prd_cus_mtrx.index, index= prd_cus_mtrx.index)

all_recomendations = {}
customers = list(set(data.CustomerID))
for customer in customers:
    top_5 = similarity.loc[customer].sort_values(ascending = False)[1:].head()
    prod_5_top = aux[aux.CustomerID.isin(top_5.index)]
    top5_products = prod_5_top.groupby(by = "ProductName").sum().sort_values(by = "Quantity", ascending = False)
    top5_products.reset_index(inplace=True)
    top5_products = top5_products[["ProductName", "Quantity"]]
    merge = pd.merge(top5_products,pd.DataFrame(prd_cus_mtrx.T[customer]), on = "ProductName")
    merge.columns = ["ProductName", "Quantity", "Purchased"]
    recomendation = merge[merge["Purchased"]==0].sort_values(by = "Quantity", ascending = False)[:5]
    all_recomendations[customer] = list(recomendation["ProductName"])
user_rec = pd.DataFrame.from_dict(all_recomendations, orient = "index").reset_index()
user_rec.columns = ["CustomerID", "Rec1", "Rec2", "Rec3", "Rec4", "Rec5"]
user_rec.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,83973,Pastry - Choclate Baked,Flour - Pastry,Puree - Passion Fruit,Lettuce - Spring Mix,Broom - Corn
1,59399,Ezy Change Mophandle,Anchovy Paste - 56 G Tube,Garlic - Peeled,Broom - Corn,Water - Aquafina Vitamin
2,92168,Wiberg Super Cure,Tuna - Salad Premix,Sage - Ground,Apricots Fresh,Beef - Inside Round
3,49159,"Turnip - White, Organic",Beer - Blue,Peas - Frozen,Lamb - Ground,Beef - Ground Medium
4,18441,V8 - Berry Blend,Whmis - Spray Bottle Trigger,Mussels - Cultivated,Ecolab - Lime - A - Way 4/4 L,Milk - 2%
